In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [22]:
# Load dataset
df = pd.read_csv("heart_disease_2000_rows.csv")

# Separate features and target
X = df.drop("Heart_Disease_Presence", axis=1)
y = df["Heart_Disease_Presence"]

# Remove duplicates for test set
df_expanded_unique = df.drop_duplicates()
print(df_expanded_unique.shape)

# Split dataset (test set stays original, unique)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)
# SCALING
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#Encode Categorical Features & Scale Numeric Features
categorical_features = ["Sex", "Chest_Pain_Type", "Fasting_Blood_Sugar", 
                        "Resting_ECG", "Exercise_Angina", "ST_Slope", 
                        "Major_Vessels", "Thalassemia"]

numeric_features = ["Age", "Resting_Blood_Pressure", "Serum_Cholesterol", 
                    "Max_Heart_Rate", "ST_Depression"]

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(drop="first"), categorical_features)
])

(302, 14)


In [23]:
df

,Age,Sex,Chest_Pain_Type,Resting_Blood_Pressure,Serum_Cholesterol,Fasting_Blood_Sugar,Resting_ECG_Results,Max_Heart_Rate_Achieved,Exercise_Induced_Angina,ST_Depression,ST_Segment_Slope,Number_of_Major_Vessels,Thalassemia,Heart_Disease_Presence
0,63,0,1,140,195,0,1,179,0,0.0,2,2,2,1
1,46,1,0,120,249,0,0,144,0,0.8,2,0,3,0
2,69,1,3,160,234,1,0,131,0,0.1,1,1,2,1
3,51,1,2,94,227,0,1,154,1,0.0,2,1,3,1
4,50,1,2,140,233,0,1,163,0,0.6,1,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,40,1,3,140,199,0,1,178,1,1.4,2,0,3,1
1996,52,1,1,128,205,1,1,184,0,0.0,2,0,2,1
1997,45,0,1,130,234,0,0,175,0,0.6,1,0,2,1
1998,58,1,1,120,284,0,0,160,0,1.8,1,0,2,0


In [24]:
# Separate classes
df_1 = df[df["Heart_Disease_Presence"] == 1]
df_0 = df[df["Heart_Disease_Presence"] == 0]

# Desired counts
n_1 = 1500   # 75%
n_0 = 500    # 25%

# Resample with replacement
df_1_resampled = df_1.sample(n=n_1, replace=True, random_state=42)
df_0_resampled = df_0.sample(n=n_0, replace=True, random_state=42)

# Combine and shuffle
df_expanded = pd.concat([df_1_resampled, df_0_resampled]) \
                 .sample(frac=1, random_state=42) \
                 .reset_index(drop=True)

print(df_expanded.shape)
print(df_expanded["Heart_Disease_Presence"].value_counts(normalize=True))

(2000, 14)
Heart_Disease_Presence
1    0.75
0    0.25
Name: proportion, dtype: float64


In [25]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# -----------------------------
# Initialize model
# -----------------------------
nb_model = GaussianNB()

# -----------------------------
# Train model
# -----------------------------
nb_model.fit(X_train, y_train)

# -----------------------------
# Predict on test data
# -----------------------------
y_pred = nb_model.predict(X_test)

# -----------------------------
# Calculate accuracy
# -----------------------------
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes Test Accuracy:", round(accuracy, 4))
print(classification_report(y_test, y_pred))

Naive Bayes Test Accuracy: 0.8325
              precision    recall  f1-score   support

           0       0.84      0.77      0.80       178
           1       0.83      0.88      0.85       222

    accuracy                           0.83       400
   macro avg       0.83      0.83      0.83       400
weighted avg       0.83      0.83      0.83       400



In [26]:
y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8225
              precision    recall  f1-score   support

           0       0.85      0.73      0.79       178
           1       0.81      0.90      0.85       222

    accuracy                           0.82       400
   macro avg       0.83      0.81      0.82       400
weighted avg       0.83      0.82      0.82       400



In [27]:
#Model 1: Logistic Regression 
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(
        C=0.01,       
        max_iter=200
    ))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred)
print("Accuracy:", acc_lr)
print(classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

intro_cmap = LinearSegmentedColormap.from_list(
    "intro_cmap",
    ["#F2FBF9", "#7ECFC0"] 
)

# Plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=intro_cmap, values_format="d")

plt.title("Confusion Matrix", fontsize=16, color="#7ECFC0")
plt.tight_layout()
plt.show()


Accuracy: 0.8225
              precision    recall  f1-score   support

           0       0.85      0.73      0.79       178
           1       0.81      0.90      0.85       222

    accuracy                           0.82       400
   macro avg       0.83      0.81      0.82       400
weighted avg       0.83      0.82      0.82       400



NameError: name 'LinearSegmentedColormap' is not defined

In [28]:
#Model 2: Logistic Regression + GridSearchCV

lr_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(
        max_iter=1000,
        solver="lbfgs"
    ))
])
param_grid = {
    "lr__C": [0.01, 0.1, 1, 10],
    "lr__penalty": ["l2"]
}

grid_lr = GridSearchCV(
    lr_pipeline,
    param_grid,
    cv=5,
    scoring="accuracy"
)
grid_lr.fit(X_train, y_train)
y_pred_grid = grid_lr.predict(X_test)
acc_grid = accuracy_score(y_test, y_pred_grid)
print("Accuracy:",acc_grid)
print(classification_report(y_test, y_pred_grid))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_grid)

intro_cmap = LinearSegmentedColormap.from_list(
    "intro_cmap",
    ["#F2FBF9", "#7ECFC0"] 
)

# Plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=intro_cmap, values_format="d")

plt.title("Confusion Matrix", fontsize=16, color="#7ECFC0")
plt.tight_layout()
plt.show()


Accuracy: 0.845
              precision    recall  f1-score   support

           0       0.85      0.79      0.82       178
           1       0.84      0.89      0.86       222

    accuracy                           0.84       400
   macro avg       0.85      0.84      0.84       400
weighted avg       0.85      0.84      0.84       400



NameError: name 'LinearSegmentedColormap' is not defined

In [9]:
#Model 3 – Gradient Descent (Logistic Regression with SGD)

# SGD Pipeline
sgd_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("sgd", SGDClassifier(
        loss="log_loss",
        penalty="l2",
        alpha=0.0001,
        max_iter=2000,
        random_state=42
    ))
])

# Train
sgd_pipeline.fit(X_train, y_train)

# Predict
y_pred_sgd = sgd_pipeline.predict(X_test)

# Metrics
acc_gd = accuracy_score(y_test, y_pred_sgd)
print("Accuracy:", acc_gd)
print(classification_report(y_test, y_pred_sgd))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_sgd)

intro_cmap = LinearSegmentedColormap.from_list(
    "intro_cmap",
    ["#F2FBF9", "#7ECFC0"] 
)

# Plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=intro_cmap, values_format="d")

plt.title("Confusion Matrix", fontsize=16, color="#7ECFC0")
plt.tight_layout()
plt.show()


Accuracy: 0.8475
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       178
           1       0.86      0.86      0.86       222

    accuracy                           0.85       400
   macro avg       0.85      0.85      0.85       400
weighted avg       0.85      0.85      0.85       400



NameError: name 'LinearSegmentedColormap' is not defined

In [10]:
#Model 4: Random Forest
rf_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", RandomForestClassifier(n_estimators=300, max_depth=6, random_state=42))
])

rf_pipeline.fit(X_train, y_train)
y_pred_rf = rf_pipeline.predict(X_test)
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy:", acc_rf)
print(classification_report(y_test, y_pred_rf))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_rf)

intro_cmap = LinearSegmentedColormap.from_list(
    "intro_cmap",
    ["#F2FBF9", "#7ECFC0"] 
)

# Plot
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=intro_cmap, values_format="d")

plt.title("Confusion Matrix", fontsize=16, color="#7ECFC0")
plt.tight_layout()
plt.show()


Accuracy: 0.97
              precision    recall  f1-score   support

           0       0.99      0.94      0.97       178
           1       0.96      0.99      0.97       222

    accuracy                           0.97       400
   macro avg       0.97      0.97      0.97       400
weighted avg       0.97      0.97      0.97       400



NameError: name 'LinearSegmentedColormap' is not defined

In [ ]:
results = pd.DataFrame({
    "Model": ["Logistic Regression", "Logistic + GridSearch","Gradient Descent", "Random Forest"],
    "Accuracy": [acc_lr, acc_grid, acc_gd, acc_rf]
})

print(results)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    log_loss,
    roc_auc_score
)

param_grid = {
    "C": [0.01, 0.1, 1, 10],
    "penalty": ["l1", "l2"],
    "solver": ["liblinear"]
}

lr = LogisticRegression(max_iter=1000)

grid = GridSearchCV(
    lr,
    param_grid,
    cv=5,
    scoring="f1",
    n_jobs=-1
)

grid.fit(X_train, y_train)

best_model = grid.best_estimator_

print("Best hyperparameters:", grid.best_params_)


Best hyperparameters: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}


In [12]:
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.85      0.79      0.82       178
           1       0.84      0.89      0.86       222

    accuracy                           0.84       400
   macro avg       0.85      0.84      0.84       400
weighted avg       0.85      0.84      0.84       400



In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# -----------------------------
# Initialize model
# -----------------------------
nb_model = GaussianNB()

# -----------------------------
# Train model
# -----------------------------
nb_model.fit(X_train, y_train)

# -----------------------------
# Predict on test data
# -----------------------------
y_pred = nb_model.predict(X_test)

# -----------------------------
# Calculate accuracy
# -----------------------------
accuracy = accuracy_score(y_test, y_pred)
print("Naive Bayes Test Accuracy:", round(accuracy, 4))

Naive Bayes Test Accuracy: 0.8325


In [14]:
# Log Loss BEFORE GridSearch (baseline model)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

lr_baseline = LogisticRegression(max_iter=1000)
lr_baseline.fit(X_train, y_train)

y_prob_base = lr_baseline.predict_proba(X_test)
logloss_base = log_loss(y_test, y_prob_base)

print("Baseline Log Loss:", logloss_base)


Baseline Log Loss: 0.36703199851868845


In [15]:
y_prob_best = best_model.predict_proba(X_test)
logloss_best = log_loss(y_test, y_prob_best)

print("Tuned Model Log Loss:", logloss_best)


Tuned Model Log Loss: 0.36684308264533044


In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss

gb = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    random_state=42
)

gb.fit(X_train, y_train)


,loss,'log_loss'
,learning_rate,0.05
,n_estimators,200
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


In [17]:
y_prob_gb = gb.predict_proba(X_test)
logloss_gb = log_loss(y_test, y_prob_gb)

print("Gradient Boosting Log Loss:", logloss_gb)


Gradient Boosting Log Loss: 0.09726876374972977


In [18]:
# Select numeric columns only
num_cols = df.select_dtypes(include=["int64", "float64"]).columns
num_cols = num_cols.drop("Heart_Disease_Presence")  # do NOT touch the target

Q1 = df[num_cols].quantile(0.25)
Q3 = df[num_cols].quantile(0.75)
IQR = Q3 - Q1

outlier_mask = ((df[num_cols] < (Q1 - 1.5 * IQR)) |
                (df[num_cols] > (Q3 + 1.5 * IQR)))

# Count outliers per column
outliers_count = outlier_mask.sum()
print(outliers_count)

Age                          0
Sex                          0
Chest_Pain_Type              0
Resting_Blood_Pressure      70
Serum_Cholesterol           30
Fasting_Blood_Sugar        301
Resting_ECG_Results          0
Max_Heart_Rate_Achieved     14
Exercise_Induced_Angina      0
ST_Depression               10
ST_Segment_Slope             0
Number_of_Major_Vessels    155
Thalassemia                 17
dtype: int64


In [19]:
# Use dataset after outlier removal
df = df_clean.copy()

X = df.drop("Heart_Disease_Presence", axis=1)
y = df["Heart_Disease_Presence"]


NameError: name 'df_clean' is not defined

In [ ]:
#Outlier Removal
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)
lr = LogisticRegression(max_iter=500)

lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

results = {
    "Before Outlier Removal": 0.8225,
    "After Outlier Removal": accuracy_score(y_test, y_pred_lr)
}

print(results)
